In [1]:
from openff.toolkit.topology import Molecule
from typing import List, Optional
import json
import copy

In [2]:
data = json.load(open("./data/MP2_heavy-aug-cc-pVTZ_torsiondrive_data.json"))

offmols = [] 
for key, value in data.items():
    offmol = Molecule.from_mapped_smiles(value["metadata"]["mapped_smiles"], allow_undefined_stereo=True)
    offmol.properties["dihedrals"] = value["metadata"]["dihedral scanned"]
    offmols.append(offmol)

In [ ]:
data = json.load(open("./data/MP2_heavy-aug-cc-pVTZ_torsiondrive_data.json"))

smiles = [] 
dihedral_atoms = []
for key, value in data.items():
    smiles.append(value["metadata"]["mapped_smiles"])
    dihedral_atoms.append(value["metadata"]["dihedral scanned"][0])
    

In [ ]:
image = oedepict.OEImage(200, 200)

rows, cols = 15, 4
grid = oedepict.OEImageGrid(image, rows, cols)

opts = oedepict.OE2DMolDisplayOptions(grid.GetCellWidth(), grid.GetCellHeight(),
                                      oedepict.OEScale_AutoScale)

for smi, cell in zip(smiles, grid.GetCells()):
    mol = oechem.OEGraphMol()
    oechem.OESmilesToMol(mol, smi)
    oedepict.OEPrepareDepiction(mol)
    disp = oedepict.OE2DMolDisplay(mol, opts)
    oedepict.OERenderMolecule(cell, disp)

oedepict.OEWriteImage("ImageGridSimple.png", image)

In [3]:
def create_openeye_pdf(molecules: List[Molecule], file_name: str, columns: int):
    """Make the pdf of the molecules using OpenEye."""

    from openeye import oechem, oedepict

    itf = oechem.OEInterface()
    suppress_h = True
    rows = 5
    cols = columns
    ropts = oedepict.OEReportOptions(rows, cols)
    ropts.SetHeaderHeight(25)
    ropts.SetFooterHeight(25)
    ropts.SetCellGap(4)
    ropts.SetPageMargins(10)
    report = oedepict.OEReport(ropts)
    cellwidth, cellheight = report.GetCellWidth(), report.GetCellHeight()
    opts = oedepict.OE2DMolDisplayOptions(
        cellwidth, cellheight, oedepict.OEScale_Default * 0.6
    )
    opts.SetAromaticStyle(oedepict.OEAromaticStyle_Circle)
    pen = oedepict.OEPen(oechem.OEBlack, oechem.OEBlack, oedepict.OEFill_On, 1.0)
    opts.SetDefaultBondPen(pen)
    oedepict.OESetup2DMolDisplayOptions(opts, itf)

    # now we load the molecules
    for ind, off_mol in enumerate(molecules):

        off_mol = copy.deepcopy(off_mol)
        off_mol._conformers = []
        off_mol.name = None

        cell = report.NewCell()
        mol = off_mol.to_openeye()
        oedepict.OEPrepareDepiction(mol, False, suppress_h)
        disp = oedepict.OE2DMolDisplay(mol, opts)

        if "dihedrals" in off_mol.properties:

            # work out if we have a double or single torsion
            if len(off_mol.properties["dihedrals"]) == 1:
                dihedrals = off_mol.properties["dihedrals"][0]
                center_bonds = dihedrals[1:3]
            else:
                # double torsion case
                dihedrals = [
                    *off_mol.properties["dihedrals"][0],
                    *off_mol.properties["dihedrals"][1],
                ]
                center_bonds = [
                    *off_mol.properties["dihedrals"][0][1:3],
                    *off_mol.properties["dihedrals"][1][1:3],
                ]

            # Highlight element of interest
            class NoAtom(oechem.OEUnaryAtomPred):
                def __call__(self, atom):
                    return False

            class AtomInTorsion(oechem.OEUnaryAtomPred):
                def __call__(self, atom):
                    return atom.GetIdx() in dihedrals

            class NoBond(oechem.OEUnaryBondPred):
                def __call__(self, bond):
                    return False

            class CentralBondInTorsion(oechem.OEUnaryBondPred):
                def __call__(self, bond):
                    return (bond.GetBgn().GetIdx() in center_bonds) and (
                        bond.GetEnd().GetIdx() in center_bonds
                    )

            atoms = mol.GetAtoms(AtomInTorsion())
            bonds = mol.GetBonds(NoBond())
            abset = oechem.OEAtomBondSet(atoms, bonds)
            oedepict.OEAddHighlighting(
                disp,
                oechem.OEColor(oechem.OELightOrange),
                oedepict.OEHighlightStyle_BallAndStick,
                abset,
            )

            atoms = mol.GetAtoms(NoAtom())
            bonds = mol.GetBonds(CentralBondInTorsion())
            abset = oechem.OEAtomBondSet(atoms, bonds)
            oedepict.OEAddHighlighting(
                disp,
                oechem.OEColor(oechem.OERedOrange),
                oedepict.OEHighlightStyle_BallAndStick,
                abset,
            )

        oedepict.OERenderMolecule(cell, disp)

    oedepict.OEWriteReport(file_name, report)


In [4]:
create_openeye_pdf(offmols, file_name='./vis.pdf', columns=4)